<a href="https://colab.research.google.com/github/shubhamjha16/jira_bug_restapi_project/blob/master/jira_pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required libraries

In [ ]:
!pip install requests pandas

2. Setup Jira Credentials

In [ ]:
import os
from google.colab import userdata

API_TOKEN = userdata.get("JIRA_TOKEN")
EMAIL = userdata.get("JIRA_EMAIL")
JIRA_URL = userdata.get("JIRA_URL")
PROJECT_KEY = userdata.get("PROJECT_KEY")

 3. Define the Bug List

In [ ]:
bugs = [
    {
        "summary": "Login form accepts empty password",
        "description": "Steps:\n1. Go to login\n2. Leave password empty\n3. Submit\nExpected: error\nActual: login"
    },
    {
        "summary": "Dark mode not applied on profile page",
        "description": "Steps:\n1. Enable dark mode\n2. Go to profile\nExpected: dark theme\nActual: light"
    },
    {
        "summary": "Search results don't update after deleting a filter",
        "description": "Steps:\n1. Search with filters\n2. Remove filter\nExpected: refreshed results\nActual: same results"
    },
    {
        "summary": "Error 500 when uploading a 10MB file",
        "description": "Steps:\n1. Upload 10MB file\n2. Submit\nExpected: success or friendly error\nActual: 500 server error"
    },
    {
        "summary": "Notification badge count doesn't reset after reading messages",
        "description": "Steps:\n1. Open new messages\nExpected: badge count = 0\nActual: still shows count"
    }
]

 Create Bugs in Jira

In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

url = f"{JIRA_URL}/rest/api/3/issue"

for bug in bugs:
    # Format description in Atlassian Document Format (ADF)
    description_adf = {
        "type": "doc",
        "version": 1,
        "content": [
            {
                "type": "paragraph",
                "content": [
                    {
                        "type": "text",
                        "text": bug["description"]
                    }
                ]
            }
        ]
    }

    payload = json.dumps({
        "fields": {
            "project": {"key": PROJECT_KEY},
            "summary": bug["summary"],
            "description": description_adf,
            "issuetype": {"name": "Bug"}
        }
    })

    response = requests.post(
        url,
        headers=headers,
        data=payload,
        auth=HTTPBasicAuth(EMAIL, API_TOKEN)
    )

    if response.status_code == 201:
        print("✅ Created:", response.json()["key"])
    else:
        print("❌ Error:", response.status_code)
        print(response.text)

✅ Created: ECS-12
✅ Created: ECS-13
✅ Created: ECS-14
✅ Created: ECS-15
✅ Created: ECS-16


Fetch All Bugs from Jira

In [ ]:
import pandas as pd

search_url = f"{JIRA_URL}/rest/api/3/search"
params = {
    "jql": f"project = {PROJECT_KEY} AND issuetype = Bug ORDER BY created DESC",
    "fields": "summary,description,created,status"
}

response = requests.get(
    search_url,
    headers={"Accept": "application/json"},
    params=params,
    auth=HTTPBasicAuth(EMAIL, API_TOKEN)
)

if response.status_code == 200:
    issues = response.json()["issues"]
    bugs_data = [{
        "Key": i["key"],
        "Summary": i["fields"]["summary"],
        "Description": i["fields"].get("description", ""),
        "Created": i["fields"]["created"],
        "Status": i["fields"]["status"]["name"]
    } for i in issues]

    df = pd.DataFrame(bugs_data)
    df.to_csv("jira_bugs.csv", index=False)
    print("✅ Saved to jira_bugs.csv")
    df.head()
else:
    print("❌ Error:", response.status_code)
    print(response.text)

✅ Saved to jira_bugs.csv


Download the CSV

In [ ]:
from google.colab import files
files.download("jira_bugs.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Install Plotly

In [ ]:
!pip install plotly

Load the CSV

In [ ]:
import pandas as pd
import plotly.express as px

# Load your bug data
df = pd.read_csv("jira_bugs.csv")

# Convert 'Created' to datetime
df["Created"] = pd.to_datetime(df["Created"])
df["Created_Date"] = df["Created"].dt.date

Bug Count by Status

In [ ]:
fig1 = px.histogram(
    df,
    x="Status",
    color="Status",
    title="🪲 Bug Count by Status",
    text_auto=True
)
fig1.show()

Bugs Over Time

In [ ]:
fig2 = px.histogram(
    df,
    x="Created_Date",
    title="📆 Bugs Created Over Time",
    nbins=20
)
fig2.update_xaxes(title="Date")
fig2.update_yaxes(title="Number of Bugs")
fig2.show()

Top Keywords in Bug Summary (Word Count)

In [ ]:
from collections import Counter
import re

# Basic word frequency in summaries
words = " ".join(df["Summary"]).lower()
words = re.findall(r'\b\w+\b', words)
word_freq = Counter(words)

top_words = pd.DataFrame(word_freq.most_common(10), columns=["Word", "Count"])

fig3 = px.bar(
    top_words,
    x="Word",
    y="Count",
    title="🔍 Most Frequent Words in Bug Summaries",
    text_auto=True
)
fig3.show()

 Interactive Dropdown Filter

In [ ]:
import ipywidgets as widgets
from IPython.display import display

status_options = sorted(df["Status"].unique())

dropdown = widgets.Dropdown(
    options=status_options,
    description="Filter by Status:"
)

def filter_table(status):
    display(df[df["Status"] == status][["Key", "Summary", "Created", "Status"]])

widgets.interact(filter_table, status=dropdown)

interactive(children=(Dropdown(description='Filter by Status:', options=('To Do',), value='To Do'), Output()),…

<function __main__.filter_table(status)>